In [1]:
import joblib
import json
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import pandas as pd


In [2]:
model = joblib.load("../src/models/logreg_final.joblib")
print(model)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['usd_goal_fx_log',
                                                   'creator_prev_projects_successful',
                                                   'creator_prev_projects',
                                                   'project_duration_days',
                                                   'blurb_len', 'name_len']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['category_name_reduced',
                                        

In [3]:
model

,steps,"[('preprocess', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [4]:
# Lataa pipeline
pipeline: Pipeline = joblib.load("../src/models/logreg_final.joblib")

preprocess: ColumnTransformer = pipeline.named_steps["preprocess"]
clf = pipeline.named_steps["clf"]

# Yritä saada featurenimet suoraan ColumnTransformerista
feature_names = preprocess.get_feature_names_out()

data = {
    "model_type": type(clf).__name__,
    "classes": clf.classes_.tolist(),           # [0, 1]
    "intercept": clf.intercept_.tolist(),       # lista per luokka (binary: 1 pituinen)
    "coefficients": [
        {"feature": name, "coef": float(coef)}
        for name, coef in zip(feature_names, clf.coef_[0])
    ],
}

json_str = json.dumps(data, ensure_ascii=False, indent=2)
print(json_str)


{
  "model_type": "LogisticRegression",
  "classes": [
    0,
    1
  ],
  "intercept": [
    -0.0009105161395630807
  ],
  "coefficients": [
    {
      "feature": "num__usd_goal_fx_log",
      "coef": -0.6702430086685636
    },
    {
      "feature": "num__creator_prev_projects_successful",
      "coef": 3.287343325110267
    },
    {
      "feature": "num__creator_prev_projects",
      "coef": -1.8163471140188114
    },
    {
      "feature": "num__project_duration_days",
      "coef": -0.22244401124375318
    },
    {
      "feature": "num__blurb_len",
      "coef": -0.05779346487496203
    },
    {
      "feature": "num__name_len",
      "coef": 0.20134676092312417
    },
    {
      "feature": "cat__category_name_reduced_Accessories",
      "coef": 0.06953271954282973
    },
    {
      "feature": "cat__category_name_reduced_Animation",
      "coef": 0.18288697327249717
    },
    {
      "feature": "cat__category_name_reduced_Apparel",
      "coef": -0.05451131182217135
    },
 

In [5]:
df = pd.DataFrame(data["coefficients"])

In [6]:
df.head()

,feature,coef
0,num__usd_goal_fx_log,-0.670243
1,num__creator_prev_projects_successful,3.287343
2,num__creator_prev_projects,-1.816347
3,num__project_duration_days,-0.222444
4,num__blurb_len,-0.057793


In [18]:
pd.set_option('display.max_rows', 200)
df.sort_values("coef", ascending=1).head(200)

,feature,coef
5826,name__canceled,-31.609769
3678,blurb__prototype,-2.050127
1618,blurb__edm,-2.020201
5134,blurb__youtube,-1.985966
5948,name__childrens,-1.940696
4969,blurb__wanting,-1.880883
2837,blurb__looking,-1.819279
2,num__creator_prev_projects,-1.816347
8630,name__promotion,-1.765063
951,blurb__childrens,-1.762462


In [14]:
df.to_csv("painot.csv")